# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f8aac379d60>
├── 'a' --> tensor([[-0.9820, -2.3604,  1.2070],
│                   [-0.6656, -0.5517, -0.6521]])
└── 'x' --> <FastTreeValue 0x7f8aac379e80>
    └── 'c' --> tensor([[ 1.0869,  2.3614, -0.4208, -0.2586],
                        [ 0.7364,  0.1505, -0.3797,  1.6764],
                        [-0.3111,  0.9107,  1.2373, -0.8553]])

In [4]:
t.a

tensor([[-0.9820, -2.3604,  1.2070],
        [-0.6656, -0.5517, -0.6521]])

In [5]:
%timeit t.a

60 ns ± 0.0494 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f8aac379d60>
├── 'a' --> tensor([[-0.7756,  0.2272, -0.2696],
│                   [ 2.4009, -1.9240, -0.9544]])
└── 'x' --> <FastTreeValue 0x7f8aac379e80>
    └── 'c' --> tensor([[ 1.0869,  2.3614, -0.4208, -0.2586],
                        [ 0.7364,  0.1505, -0.3797,  1.6764],
                        [-0.3111,  0.9107,  1.2373, -0.8553]])

In [7]:
%timeit t.a = new_value

61.9 ns ± 0.0319 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.9820, -2.3604,  1.2070],
               [-0.6656, -0.5517, -0.6521]]),
    x: Batch(
           c: tensor([[ 1.0869,  2.3614, -0.4208, -0.2586],
                      [ 0.7364,  0.1505, -0.3797,  1.6764],
                      [-0.3111,  0.9107,  1.2373, -0.8553]]),
       ),
)

In [10]:
b.a

tensor([[-0.9820, -2.3604,  1.2070],
        [-0.6656, -0.5517, -0.6521]])

In [11]:
%timeit b.a

54.1 ns ± 0.029 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 1.4289, -0.5579, -1.6722],
               [-1.5345, -0.5444,  1.2258]]),
    x: Batch(
           c: tensor([[ 1.0869,  2.3614, -0.4208, -0.2586],
                      [ 0.7364,  0.1505, -0.3797,  1.6764],
                      [-0.3111,  0.9107,  1.2373, -0.8553]]),
       ),
)

In [13]:
%timeit b.a = new_value

478 ns ± 0.356 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

842 ns ± 0.294 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.6 µs ± 39.9 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

136 µs ± 1.19 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

137 µs ± 647 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f8aac38de50>
├── 'a' --> tensor([[[-0.9820, -2.3604,  1.2070],
│                    [-0.6656, -0.5517, -0.6521]],
│           
│                   [[-0.9820, -2.3604,  1.2070],
│                    [-0.6656, -0.5517, -0.6521]],
│           
│                   [[-0.9820, -2.3604,  1.2070],
│                    [-0.6656, -0.5517, -0.6521]],
│           
│                   [[-0.9820, -2.3604,  1.2070],
│                    [-0.6656, -0.5517, -0.6521]],
│           
│                   [[-0.9820, -2.3604,  1.2070],
│                    [-0.6656, -0.5517, -0.6521]],
│           
│                   [[-0.9820, -2.3604,  1.2070],
│                    [-0.6656, -0.5517, -0.6521]],
│           
│                   [[-0.9820, -2.3604,  1.2070],
│                    [-0.6656, -0.5517, -0.6521]],
│           
│                   [[-0.9820, -2.3604,  1.2070],
│                    [-0.6656, -0.5517, -0.6521]]])
└── 'x' --> <FastTreeValue 0x7f8aac38d550>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32 µs ± 127 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f8a0101bdf0>
├── 'a' --> tensor([[-0.9820, -2.3604,  1.2070],
│                   [-0.6656, -0.5517, -0.6521],
│                   [-0.9820, -2.3604,  1.2070],
│                   [-0.6656, -0.5517, -0.6521],
│                   [-0.9820, -2.3604,  1.2070],
│                   [-0.6656, -0.5517, -0.6521],
│                   [-0.9820, -2.3604,  1.2070],
│                   [-0.6656, -0.5517, -0.6521],
│                   [-0.9820, -2.3604,  1.2070],
│                   [-0.6656, -0.5517, -0.6521],
│                   [-0.9820, -2.3604,  1.2070],
│                   [-0.6656, -0.5517, -0.6521],
│                   [-0.9820, -2.3604,  1.2070],
│                   [-0.6656, -0.5517, -0.6521],
│                   [-0.9820, -2.3604,  1.2070],
│                   [-0.6656, -0.5517, -0.6521]])
└── 'x' --> <FastTreeValue 0x7f8a0101bd90>
    └── 'c' --> tensor([[ 1.0869,  2.3614, -0.4208, -0.2586],
                        [ 0.7364,  0.1505, -0.3797,  1.6764],
                 

In [23]:
%timeit t_cat(trees)

30.1 µs ± 42.2 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

56.4 µs ± 92.1 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-0.9820, -2.3604,  1.2070],
                [-0.6656, -0.5517, -0.6521]],
       
               [[-0.9820, -2.3604,  1.2070],
                [-0.6656, -0.5517, -0.6521]],
       
               [[-0.9820, -2.3604,  1.2070],
                [-0.6656, -0.5517, -0.6521]],
       
               [[-0.9820, -2.3604,  1.2070],
                [-0.6656, -0.5517, -0.6521]],
       
               [[-0.9820, -2.3604,  1.2070],
                [-0.6656, -0.5517, -0.6521]],
       
               [[-0.9820, -2.3604,  1.2070],
                [-0.6656, -0.5517, -0.6521]],
       
               [[-0.9820, -2.3604,  1.2070],
                [-0.6656, -0.5517, -0.6521]],
       
               [[-0.9820, -2.3604,  1.2070],
                [-0.6656, -0.5517, -0.6521]]]),
    x: Batch(
           c: tensor([[[ 1.0869,  2.3614, -0.4208, -0.2586],
                       [ 0.7364,  0.1505, -0.3797,  1.6764],
                       [-0.3111,  0.9107,  1.2373, -0.8553]],
         

In [26]:
%timeit Batch.stack(batches)

77.6 µs ± 91.6 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-0.9820, -2.3604,  1.2070],
               [-0.6656, -0.5517, -0.6521],
               [-0.9820, -2.3604,  1.2070],
               [-0.6656, -0.5517, -0.6521],
               [-0.9820, -2.3604,  1.2070],
               [-0.6656, -0.5517, -0.6521],
               [-0.9820, -2.3604,  1.2070],
               [-0.6656, -0.5517, -0.6521],
               [-0.9820, -2.3604,  1.2070],
               [-0.6656, -0.5517, -0.6521],
               [-0.9820, -2.3604,  1.2070],
               [-0.6656, -0.5517, -0.6521],
               [-0.9820, -2.3604,  1.2070],
               [-0.6656, -0.5517, -0.6521],
               [-0.9820, -2.3604,  1.2070],
               [-0.6656, -0.5517, -0.6521]]),
    x: Batch(
           c: tensor([[ 1.0869,  2.3614, -0.4208, -0.2586],
                      [ 0.7364,  0.1505, -0.3797,  1.6764],
                      [-0.3111,  0.9107,  1.2373, -0.8553],
                      [ 1.0869,  2.3614, -0.4208, -0.2586],
                      [ 0.7364,  

In [28]:
%timeit Batch.cat(batches)

142 µs ± 143 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

314 µs ± 2.17 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
